# Equal-Weight S&P 500 Index Fund.

In [ ]:
# Import libraries

import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [ ]:
# Import list of stocks

stocks = pd.read_csv("sp_500_stocks.csv")
stocks = stocks.drop([135,219, 467, 484]) # stock giving error, might not be in list

In [ ]:
# Import IEX Cloud API Token (Sandbox API)

from secret import IEX_CLOUD_API_TOKEN
token = IEX_CLOUD_API_TOKEN

In [ ]:
# Make first API call

symbol = "CME"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={token}"
data = requests.get(api_url).json()
# data

In [ ]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of shares to buy']
final_df = pd.DataFrame(columns = my_columns)

for stock in stocks["Ticker"]:
    
    # Make API Call for each stock
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={token}"
    response = requests.get(api_url)
    response.raise_for_status()  # raises exception when not a 2xx response
    if response.status_code != 204:
        data = response.json()
         

    # Parse API Call
    price = data['latestPrice']
    market_cap = data['marketCap']
    
    # Add stocks to dataframe
    new_stock = pd.DataFrame([[stock, price, market_cap, "N/A"]], columns = my_columns)
    final_df = pd.concat([final_df, new_stock])

In [ ]:
final_df

## Using Batch API Calls to improve performance.

In [ ]:
def divide_chunks(lst, n):     
    """Yield successive n sized chunks/batches from lst"""
    for i in range(0, len(lst), n):
        yield lst.loc[i:i + n]

In [ ]:
symbol_groups = list(divide_chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range (0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    # Make Batch API Call 
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={token}"
    response = requests.get(batch_api_call_url)
    response.raise_for_status()  # raises exception when not a 2xx response
    if response.status_code != 204:
        data = response.json()
    
    for symbol in symbol_string.split(','):
        
        if symbol in data.keys():
            # Parse API Call
            price = data[symbol]['quote']['latestPrice']
            market_cap = data[symbol]['quote']['marketCap']

            # Add stocks to dataframe
            new_stock = pd.DataFrame([[symbol, price, market_cap, "N/A"]], columns = my_columns)
            final_df = pd.concat([final_df, new_stock])

In [ ]:
final_df = final_df.set_index("Ticker").reset_index()
final_df

## Calculating the number of shares to buy.

In [ ]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("Please enter an integer.")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

In [ ]:
position_size = val/len(final_df.index)
final_df['Number of shares to buy'] = (position_size/final_df['Stock Price']).apply(np.floor)
final_df

## Formatting Excel Output

In [ ]:
# Initialize xlsxwriter object
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_df.to_excel(writer, 'Recommended Trades', index = False)


In [ ]:
# Set formats
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

dollar_format = writer.book.add_format(
    {
        "num_format": "$0.00",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

integer_format = writer.book.add_format(
    {
        "num_format": "0",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

In [ ]:
# Apply formats
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format], 
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of shares to buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f"{column}:{column}", 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f"{column}1", column_formats[column][0], column_formats[column][1])

In [ ]:
# Save .xlsx file
writer.save()